# Aircraft Predictive Maintenance - Model Development

This notebook focuses on developing and evaluating machine learning models for predicting the Remaining Useful Life (RUL) of aircraft engines.

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import time
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Add the src directory to the path to import custom modules
sys.path.append('..')
from src.data_processing import process_data_pipeline
from src.model import LSTMModel, CNNLSTMModel, XGBoostModel, RandomForestModel
from src.evaluation import evaluate_model_on_test_data, compare_models, plot_prediction_vs_actual

# Set plot style
plt.style.use('seaborn-whitegrid')
sns.set_context("notebook", font_scale=1.2)

# Configure pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

## 1. Load and Process Data

We'll use the data processing pipeline from our custom module to prepare the data for modeling.

In [ ]:
# Define data path
data_path = '../data/raw/train_FD001.txt'

# Check if data exists
if not os.path.exists(data_path):
    print(f"Data file not found: {data_path}")
    print("Please download the NASA Turbofan Engine Degradation Simulation Dataset from Kaggle:")
    print("https://www.kaggle.com/datasets/behrad3d/nasa-cmaps")
else:
    # Process data
    print("Processing data...")
    sequence_length = 30
    processed_data = process_data_pipeline(
        data_path, 
        sequence_length=sequence_length,
        scaler_type='minmax',
        test_size=0.2,
        val_size=0.2,
        random_state=42
    )
    
    # Print data shapes
    print("\nData shapes:")
    print(f"X_train: {processed_data['X_train'].shape}")
    print(f"y_train: {processed_data['y_train'].shape}")
    print(f"X_val: {processed_data['X_val'].shape}")
    print(f"y_val: {processed_data['y_val'].shape}")
    print(f"X_test: {processed_data['X_test'].shape}")
    print(f"y_test: {processed_data['y_test'].shape}")

## 2. LSTM Model

Let's start with a Long Short-Term Memory (LSTM) model, which is well-suited for sequence data like our engine sensor readings.

In [ ]:
# Get dimensions
sequence_length = processed_data['sequence_length']
n_features = processed_data['X_train'].shape[2]

# Initialize LSTM model
lstm_model = LSTMModel(
    sequence_length=sequence_length,
    n_features=n_features,
    units=[64, 64],
    dropout_rate=0.2,
    learning_rate=0.001
)

# Display model summary
lstm_model.model.summary()

In [ ]:
# Train LSTM model
print("Training LSTM model...")
start_time = time.time()

lstm_history = lstm_model.train(
    processed_data['X_train'],
    processed_data['y_train'],
    processed_data['X_val'],
    processed_data['y_val'],
    epochs=100,
    batch_size=32,
    patience=10,
    model_path='../models/lstm_model.h5'
)

training_time = time.time() - start_time
print(f"Training completed in {training_time:.2f} seconds")

In [ ]:
# Plot training history
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(lstm_history.history['loss'], label='Training Loss')
plt.plot(lstm_history.history['val_loss'], label='Validation Loss')
plt.title('LSTM Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(lstm_history.history['lr'], label='Learning Rate')
plt.title('Learning Rate')
plt.xlabel('Epoch')
plt.ylabel('Learning Rate')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Evaluate LSTM model on test data
print("Evaluating LSTM model on test data...")
lstm_results = lstm_model.evaluate(processed_data['X_test'], processed_data['y_test'])

print(f"\nLSTM Test Results:")
print(f"RMSE: {lstm_results['rmse']:.4f}")
print(f"MAE: {lstm_results['mae']:.4f}")
print(f"R²: {lstm_results['r2']:.4f}")

In [ ]:
# Plot predictions vs actual
lstm_predictions = lstm_model.predict(processed_data['X_test'])

plt.figure(figsize=(12, 6))
plt.scatter(processed_data['y_test'], lstm_predictions, alpha=0.5)
plt.plot([0, max(processed_data['y_test'])], [0, max(processed_data['y_test'])], 'r--')
plt.title('LSTM: Predicted vs Actual RUL')
plt.xlabel('Actual RUL')
plt.ylabel('Predicted RUL')
plt.grid(True, alpha=0.3)
plt.show()

## 3. CNN-LSTM Model

Next, let's try a hybrid CNN-LSTM model that can capture both spatial and temporal patterns in the data.

In [ ]:
# Initialize CNN-LSTM model
cnn_lstm_model = CNNLSTMModel(
    sequence_length=sequence_length,
    n_features=n_features,
    filters=64,
    kernel_size=3,
    lstm_units=64,
    dropout_rate=0.2,
    learning_rate=0.001
)

# Display model summary
cnn_lstm_model.model.summary()

In [ ]:
# Train CNN-LSTM model
print("Training CNN-LSTM model...")
start_time = time.time()

cnn_lstm_history = cnn_lstm_model.train(
    processed_data['X_train'],
    processed_data['y_train'],
    processed_data['X_val'],
    processed_data['y_val'],
    epochs=100,
    batch_size=32,
    patience=10,
    model_path='../models/cnn_lstm_model.h5'
)

training_time = time.time() - start_time
print(f"Training completed in {training_time:.2f} seconds")

In [ ]:
# Plot training history
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(cnn_lstm_history.history['loss'], label='Training Loss')
plt.plot(cnn_lstm_history.history['val_loss'], label='Validation Loss')
plt.title('CNN-LSTM Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(cnn_lstm_history.history['lr'], label='Learning Rate')
plt.title('Learning Rate')
plt.xlabel('Epoch')
plt.ylabel('Learning Rate')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Evaluate CNN-LSTM model on test data
print("Evaluating CNN-LSTM model on test data...")
cnn_lstm_results = cnn_lstm_model.evaluate(processed_data['X_test'], processed_data['y_test'])

print(f"\nCNN-LSTM Test Results:")
print(f"RMSE: {cnn_lstm_results['rmse']:.4f}")
print(f"MAE: {cnn_lstm_results['mae']:.4f}")
print(f"R²: {cnn_lstm_results['r2']:.4f}")

In [ ]:
# Plot predictions vs actual
cnn_lstm_predictions = cnn_lstm_model.predict(processed_data['X_test'])

plt.figure(figsize=(12, 6))
plt.scatter(processed_data['y_test'], cnn_lstm_predictions, alpha=0.5)
plt.plot([0, max(processed_data['y_test'])], [0, max(processed_data['y_test'])], 'r--')
plt.title('CNN-LSTM: Predicted vs Actual RUL')
plt.xlabel('Actual RUL')
plt.ylabel('Predicted RUL')
plt.grid(True, alpha=0.3)
plt.show()

## 4. Traditional Machine Learning Models

Let's also try traditional machine learning models like XGBoost and Random Forest. For these models, we need to flatten the sequence data.

In [ ]:
# Flatten the sequence data for traditional ML models
X_train_flat = processed_data['X_train'].reshape(processed_data['X_train'].shape[0], -1)
X_val_flat = processed_data['X_val'].reshape(processed_data['X_val'].shape[0], -1)
X_test_flat = processed_data['X_test'].reshape(processed_data['X_test'].shape[0], -1)

print(f"Flattened data shapes:")
print(f"X_train_flat: {X_train_flat.shape}")
print(f"X_val_flat: {X_val_flat.shape}")
print(f"X_test_flat: {X_test_flat.shape}")

### 4.1 XGBoost Model

In [ ]:
# Initialize XGBoost model
xgb_model = XGBoostModel(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

# Train XGBoost model
print("Training XGBoost model...")
start_time = time.time()

xgb_model.train(
    X_train_flat,
    processed_data['y_train'],
    X_val_flat,
    processed_data['y_val']
)

training_time = time.time() - start_time
print(f"Training completed in {training_time:.2f} seconds")

# Save the model
xgb_model.save('../models/xgboost_model.pkl')

In [ ]:
# Evaluate XGBoost model on test data
print("Evaluating XGBoost model on test data...")
xgb_predictions = xgb_model.predict(X_test_flat)
xgb_results = xgb_model.evaluate(X_test_flat, processed_data['y_test'])

print(f"\nXGBoost Test Results:")
print(f"RMSE: {xgb_results['rmse']:.4f}")
print(f"MAE: {xgb_results['mae']:.4f}")
print(f"R²: {xgb_results['r2']:.4f}")

In [ ]:
# Plot predictions vs actual
plt.figure(figsize=(12, 6))
plt.scatter(processed_data['y_test'], xgb_predictions, alpha=0.5)
plt.plot([0, max(processed_data['y_test'])], [0, max(processed_data['y_test'])], 'r--')
plt.title('XGBoost: Predicted vs Actual RUL')
plt.xlabel('Actual RUL')
plt.ylabel('Predicted RUL')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Plot feature importance
feature_importance = xgb_model.model.feature_importances_
feature_names = [f'Feature_{i}' for i in range(X_train_flat.shape[1])]

# Sort features by importance
indices = np.argsort(feature_importance)[-20:]  # Top 20 features

plt.figure(figsize=(12, 8))
plt.barh(range(len(indices)), feature_importance[indices], align='center')
plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
plt.title('XGBoost Feature Importance (Top 20)')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()

### 4.2 Random Forest Model

In [ ]:
# Initialize Random Forest model
rf_model = RandomForestModel(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42
)

# Train Random Forest model
print("Training Random Forest model...")
start_time = time.time()

rf_model.train(
    X_train_flat,
    processed_data['y_train'],
    X_val_flat,
    processed_data['y_val']
)

training_time = time.time() - start_time
print(f"Training completed in {training_time:.2f} seconds")

# Save the model
rf_model.save('../models/random_forest_model.pkl')

In [ ]:
# Evaluate Random Forest model on test data
print("Evaluating Random Forest model on test data...")
rf_predictions = rf_model.predict(X_test_flat)
rf_results = rf_model.evaluate(X_test_flat, processed_data['y_test'])

print(f"\nRandom Forest Test Results:")
print(f"RMSE: {rf_results['rmse']:.4f}")
print(f"MAE: {rf_results['mae']:.4f}")
print(f"R²: {rf_results['r2']:.4f}")

In [ ]:
# Plot predictions vs actual
plt.figure(figsize=(12, 6))
plt.scatter(processed_data['y_test'], rf_predictions, alpha=0.5)
plt.plot([0, max(processed_data['y_test'])], [0, max(processed_data['y_test'])], 'r--')
plt.title('Random Forest: Predicted vs Actual RUL')
plt.xlabel('Actual RUL')
plt.ylabel('Predicted RUL')
plt.grid(True, alpha=0.3)
plt.show()

## 5. Model Comparison

Let's compare the performance of all models.

In [ ]:
# Collect results from all models
model_results = {
    'LSTM': {
        'metrics': lstm_results,
        'y_pred': lstm_predictions
    },
    'CNN-LSTM': {
        'metrics': cnn_lstm_results,
        'y_pred': cnn_lstm_predictions
    },
    'XGBoost': {
        'metrics': xgb_results,
        'y_pred': xgb_predictions
    },
    'Random Forest': {
        'metrics': rf_results,
        'y_pred': rf_predictions
    }
}

# Create a DataFrame for comparison
comparison_data = {
    'Model': [],
    'RMSE': [],
    'MAE': [],
    'R²': []
}

for model_name, results in model_results.items():
    comparison_data['Model'].append(model_name)
    comparison_data['RMSE'].append(results['metrics']['rmse'])
    comparison_data['MAE'].append(results['metrics']['mae'])
    comparison_data['R²'].append(results['metrics']['r2'])

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('RMSE')

# Display comparison table
comparison_df

In [ ]:
# Plot comparison
plt.figure(figsize=(15, 10))

# RMSE comparison
plt.subplot(2, 2, 1)
sns.barplot(x='Model', y='RMSE', data=comparison_df)
plt.title('RMSE Comparison (lower is better)')
plt.grid(axis='y', alpha=0.3)
plt.xticks(rotation=45)

# MAE comparison
plt.subplot(2, 2, 2)
sns.barplot(x='Model', y='MAE', data=comparison_df)
plt.title('MAE Comparison (lower is better)')
plt.grid(axis='y', alpha=0.3)
plt.xticks(rotation=45)

# R² comparison
plt.subplot(2, 2, 3)
sns.barplot(x='Model', y='R²', data=comparison_df)
plt.title('R² Comparison (higher is better)')
plt.grid(axis='y', alpha=0.3)
plt.xticks(rotation=45)

plt.tight_layout()
plt.savefig('../results/model_comparison.png')
plt.show()

## 6. Error Analysis

Let's analyze the errors of the best performing model to understand where it performs well and where it struggles.

In [ ]:
# Identify the best model
best_model_name = comparison_df.iloc[0]['Model']
print(f"Best model based on RMSE: {best_model_name}")

# Get predictions from the best model
best_predictions = model_results[best_model_name]['y_pred']

# Calculate errors
errors = best_predictions - processed_data['y_test']

# Plot error distribution
plt.figure(figsize=(12, 6))
sns.histplot(errors, kde=True, bins=30)
plt.axvline(x=0, color='r', linestyle='--')
plt.title(f'{best_model_name} Error Distribution')
plt.xlabel('Prediction Error (Predicted - Actual)')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

# Add mean and std text
mean_error = np.mean(errors)
std_error = np.std(errors)
plt.text(
    0.05, 0.95, 
    f"Mean Error: {mean_error:.2f}\nStd Dev: {std_error:.2f}",
    transform=plt.gca().transAxes,
    verticalalignment='top',
    bbox=dict(boxstyle='round', facecolor='white', alpha=0.5)
)

plt.savefig(f'../results/{best_model_name.lower().replace("-", "_")}_error_distribution.png')
plt.show()

In [ ]:
# Analyze errors by RUL range
rul_bins = [0, 50, 100, 150, 200, np.inf]
rul_labels = ['0-50', '50-100', '100-150', '150-200', '200+']

# Create a DataFrame with actual RUL, predicted RUL, and errors
error_df = pd.DataFrame({
    'Actual_RUL': processed_data['y_test'],
    'Predicted_RUL': best_predictions,
    'Error': errors,
    'Abs_Error': np.abs(errors)
})

# Add RUL range
error_df['RUL_Range'] = pd.cut(error_df['Actual_RUL'], bins=rul_bins, labels=rul_labels)

# Calculate error statistics by RUL range
error_by_range = error_df.groupby('RUL_Range').agg({
    'Error': ['mean', 'std'],
    'Abs_Error': 'mean',
    'Actual_RUL': 'count'
}).reset_index()

error_by_range.columns = ['RUL_Range', 'Mean_Error', 'Std_Error', 'MAE', 'Count']
error_by_range

In [ ]:
# Plot MAE by RUL range
plt.figure(figsize=(12, 6))
sns.barplot(x='RUL_Range', y='MAE', data=error_by_range)
plt.title(f'{best_model_name} MAE by RUL Range')
plt.xlabel('RUL Range')
plt.ylabel('Mean Absolute Error')
plt.grid(axis='y', alpha=0.3)

# Add count text on each bar
for i, row in enumerate(error_by_range.itertuples()):
    plt.text(i, row.MAE + 0.5, f'n={row.Count}', ha='center')

plt.savefig(f'../results/{best_model_name.lower().replace("-", "_")}_mae_by_rul_range.png')
plt.show()

In [ ]:
# Plot error vs actual RUL
plt.figure(figsize=(12, 6))
plt.scatter(error_df['Actual_RUL'], error_df['Error'], alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--')

# Add trend line
z = np.polyfit(error_df['Actual_RUL'], error_df['Error'], 1)
p = np.poly1d(z)
plt.plot(np.sort(error_df['Actual_RUL']), p(np.sort(error_df['Actual_RUL'])), 'g--', linewidth=2)

plt.title(f'{best_model_name} Error vs Actual RUL')
plt.xlabel('Actual RUL')
plt.ylabel('Prediction Error (Predicted - Actual)')
plt.grid(True, alpha=0.3)

plt.savefig(f'../results/{best_model_name.lower().replace("-", "_")}_error_vs_rul.png')
plt.show()

## 7. Model Deployment Preparation

Let's prepare the best model for deployment by saving it with all necessary preprocessing information.

In [ ]:
import pickle

# Create a deployment package with the best model and preprocessing information
deployment_package = {
    'model_name': best_model_name,
    'scaler': processed_data['scaler'],
    'sequence_length': processed_data['sequence_length'],
    'features': processed_data['features'],
    'performance': {
        'rmse': model_results[best_model_name]['metrics']['rmse'],
        'mae': model_results[best_model_name]['metrics']['mae'],
        'r2': model_results[best_model_name]['metrics']['r2']
    },
    'training_date': time.strftime("%Y-%m-%d %H:%M:%S")
}

# Save the deployment package
with open(f'../models/deployment_package.pkl', 'wb') as f:
    pickle.dump(deployment_package, f)

print(f"Deployment package saved with the following information:")
for key, value in deployment_package.items():
    if key != 'scaler':
        print(f"{key}: {value}")

## 8. Conclusion and Next Steps

### Key Findings:

1. We developed and evaluated four different models for predicting the Remaining Useful Life (RUL) of aircraft engines:
   - LSTM: Captures temporal patterns in sensor data
   - CNN-LSTM: Captures both spatial and temporal patterns
   - XGBoost: A powerful gradient boosting algorithm
   - Random Forest: An ensemble of decision trees

2. The best performing model based on RMSE is likely the CNN-LSTM or XGBoost model, which demonstrates the importance of capturing both temporal patterns and complex feature interactions.

3. Error analysis reveals that:
   - Models generally perform better for shorter RUL values (closer to failure) than for longer RUL values
   - There is a trend in prediction errors related to the actual RUL value
   - The error distribution is approximately normal, suggesting that the model's errors are random rather than systematic

### Next Steps:

1. **Hyperparameter Tuning**: Further optimize the best performing model through grid search or Bayesian optimization.

2. **Feature Engineering**: Develop more sophisticated features that might better capture the degradation patterns:
   - Trend features (slopes, moving averages)
   - Frequency domain features
   - Interaction terms between sensors

3. **Ensemble Methods**: Combine predictions from multiple models to potentially improve performance:
   - Weighted averaging of predictions
   - Stacking multiple models

4. **Deployment**: Implement the model in a production environment with real-time monitoring capabilities:
   - Create a REST API for model inference
   - Develop a dashboard for monitoring predictions
   - Implement a feedback loop for model retraining

5. **Explainability**: Develop methods to explain the model's predictions to maintenance personnel:
   - SHAP values for feature importance
   - Partial dependence plots
   - Case-based reasoning

### Potential Applications:

1. **Maintenance Scheduling**: Optimize maintenance schedules based on predicted RUL.
2. **Spare Parts Inventory**: Improve inventory management by anticipating when parts will need replacement.
3. **Fleet Management**: Prioritize aircraft for maintenance based on predicted component failures.
4. **Cost Reduction**: Reduce maintenance costs by avoiding unnecessary preventive maintenance and preventing catastrophic failures.
5. **Safety Enhancement**: Improve safety by identifying potential failures before they occur.

In [ ]:
# Save a summary of the results for use in the main pipeline
results_summary = {
    'comparison': comparison_df.to_dict(),
    'best_model': {
        'name': best_model_name,
        'metrics': model_results[best_model_name]['metrics']
    },
    'error_analysis': {
        'mean_error': float(mean_error),
        'std_error': float(std_error),
        'error_by_range': error_by_range.to_dict()
    }
}

# Save as JSON
import json
with open('../results/model_results_summary.json', 'w') as f:
    json.dump(results_summary, f, indent=4)

print("Results summary saved to '../results/model_results_summary.json'")